# Blind Spot Detection (BSD) with Active Intervention

*Project for the course Smart Vehicular Systems (Academic Year 2024/2025).*

**Name:** Pablo Sebastian

**Surname:** Vargas Grateron

**Email:** pablo.vargasgrateron@studio.unibo.it

This project implements a blind spot detection system using CARLA simulator and MQTT for communication. The system detects vehicles in the blind spot of a simulated car and can trigger an active intervention if necessary.

> Before running the code, ensure that the CARLA server is running and the MQTT broker is accessible.

## Libraries and setup

In [ ]:
import carla, time, pygame, math, random, cv2
import numpy as np
import paho.mqtt.client as mqtt

This cell configures the CARLA server connection.

In [ ]:
carla_client = carla.Client('localhost', 2000)
carla_client.set_timeout(10.0)

world = carla_client.get_world()
spectator = world.get_spectator()

This cell configures the MQTT client.

In [ ]:
BROKER = "test.mosquitto.org"
PORT = 1883
TOPIC = "svs_blind_spot_detection"

mqtt_client = mqtt.Client()
mqtt_client.connect(BROKER, PORT, 60)

Further setup for the MQTT client, including connection and message handling.

In [ ]:
def mqtt_on_connect(client, userdata, flags, rc):
    print(f"[MQTT] Connected (code {rc})")
    client.subscribe(TOPIC)

def mqtt_on_message(client, userdata, message):
    print(f"[MQTT] Message on {message.topic} -> {message.payload.decode()}")

def mqtt_send_message(topic, message):
    mqtt_client.publish(topic, message)
    print(f"[MQTT] Sent message: {message} to topic: {topic}")

mqtt_client.on_connect = mqtt_on_connect
mqtt_client.on_message = mqtt_on_message

mqtt_client.loop_start()

## Functions

### CARLA basic functions

In [ ]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

### Car image detection functions

We will implement functions to capture images from the car's cameras and detect vehicles using YOLOv5.

### Sensors functions

The next cells implement the functions to spawn:

- The **camera sensors** to capture image to further process with YOLOv5 and detect vehicles.
- The **radar sensor** to detect vehicles in the blind spot.

In [ ]:
def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=0.0, y=0.0, z=1.0), carla.Rotation(pitch=0)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None, transform=carla.Transform(carla.Location(x=0.0, y=0.0, z=1.0), carla.Rotation(pitch=0)), range=100, horizontal_fov=30, vertical_fov=30):
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', str(horizontal_fov))
    radar_bp.set_attribute('vertical_fov', str(vertical_fov))
    radar_bp.set_attribute('range', str(range))
    radar = world.spawn_actor(radar_bp, transform, attach_to=attach_to)
    return radar

## Main program and code execution